<details>
<summary>Table of Contents</summary>

- [1. Pretrain](#1-pretrain)
- [2. Finetune](#2-finetune)

</details>



This notebook demonstrates experiments on 5 countries with self-supervised learning. 
All parameters and settings are taken from the paper.

In [2]:
import pandas as pd
import subprocess
import os
from utils.helper import extract_metrics_from_output

# 1. Pretrain 

Tracking loss - MAE.

In [ ]:
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
pred_lens = ["24", "96", "168"]

for country in countries:
    for pred_len in pred_lens:
        # Just use the same parameters as from the paper
        params = {
            "--dset": country,
            "--mask_ratio": 0.4,
            "--scaler_type": "minmax",
            "--n_epochs_pretrain": 100,
            "--target_points": pred_len
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_pretrain.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        # Execute the command
        print(f"==========Running command for {country}, pred_len {pred_len}:==========")
        !{command}


==========Running command for DE, pred_len 24:==========
args: Namespace(dset_pretrain='DE', context_points=512, target_points=24, batch_size=64, num_workers=0, features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=512, dropout=0.2, head_dropout=0.2, mask_ratio=0.4, n_epochs_pretrain=100, lr=0.0001, pretrained_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
suggested_lr 0.00035938136638046257
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 603404
          epoch     train_loss     valid_loss           time
Better model found at epoch 0 with valid_loss value: 0.9651524109799743.
              0       0.982107       0.965152          00:17
Better model found at epoch 1 with valid_loss value: 0.9606560137526138.
              1       0.965265       0.960656      

# 2. Finetune

Linear probing + full fine-tuning.

In [3]:
# 2 hrs
# In case you just want finetune, please uncomment below
countries = ['DE', 'GB', 'ES', 'FR', 'IT']
pred_lens = ["24", "96", "168"]

# List to store metrics for the DataFrame
data = []

for country in countries:
    for pred_len in pred_lens:
        # Define the parameters for each country and pred_len
        params = {
            "--dset": country,
            "--is_finetune": 1,
            "--pretrained_model": f"saved_models/{country}/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth",
            "--scaler_type": "minmax",
            "--n_epochs_finetune": 20,
            "--target_points": pred_len  
        }

        # Build the command string
        command = "python PatchTST-main/PatchTST_self_supervised/patchtst_finetune.py "

        # Add parameters to the command
        for key, value in params.items():
            if value is not None:
                command += f"{key} {value} "
            else:
                command += f"{key} "  # Add flags with no value

        print(f"==========Running command for {country}, pred_len {pred_len}:==========")

        # Run the command and capture the output
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

        # Capture the output in real-time
        output = []
        for line in process.stdout:
            output.append(line)
            print(line, end='')  # Print in the .ipynb cell

        # Wait for the process to complete
        process.stdout.close()
        process.wait()

        try:
            # Extract metrics from the output using the function
            metrics = extract_metrics_from_output(output, if_supervised=False)
            mse, rmse, mae = metrics[0]

            # Append the results to the data list
            data.append({
                'Country': country,
                'Pred_len': pred_len,
                'MSE': mse,
                'RMSE': rmse,
                'MAE': mae
            })
        except ValueError as e:
            print(f"\nError: {e} for {country}, pred_len {pred_len}")

==========Running command for DE, pred_len 24:==========
args: Namespace(is_finetune=1, is_linear_probe=0, dset_finetune='DE', context_points=512, target_points=24, batch_size=64, num_workers=0, scaler='standard', features='M', patch_len=12, stride=12, revin=1, overlapping_windows=True, scaler_type='minmax', if_relu=False, n_layers=3, n_heads=16, d_model=128, d_ff=256, dropout=0.2, head_dropout=0.2, n_epochs_finetune=20, lr=0.0001, pretrained_model='saved_models/DE/masked_patchtst/based_model/patchtst_pretrained_cw512_patch12_stride12_epochs-pretrain100_mask0.4_model1.pth', finetuned_model_id=1, model_type='based_model')
scaler_type: minmax overlapping_windows: True
number of patches: 42
number of model params 533528
check unmatched_layers: ['backbone.encoder.layers.0.ff.0.weight', 'backbone.encoder.layers.0.ff.0.bias', 'backbone.encoder.layers.0.ff.3.weight', 'backbone.encoder.layers.1.ff.0.weight', 'backbone.encoder.layers.1.ff.0.bias', 'backbone.encoder.layers.1.ff.3.weight', 'backb

In [4]:
df_self_sup = pd.DataFrame(data)
df_self_sup.set_index(['Country', 'Pred_len'], inplace=True)
df_self_sup.columns = pd.MultiIndex.from_product([['Self-supervised'], ['MSE','RMSE', 'MAE']], names=['Model', 'Metrics'])

# Path if not exists
path = "results/patchtst/"
if not os.path.exists(path):
    os.makedirs(path)

# Save the results to a CSV file
df_self_sup.to_csv(os.path.join(path, "finetune_results.csv"), index=True)
df_self_sup.round(4)

Model            Self-supervised                
Metrics                      MSE    RMSE     MAE
Country Pred_len                                
DE      24                0.0210  0.1449  0.0884
        96                0.0350  0.1872  0.1253
        168               0.0379  0.1947  0.1313
GB      24                0.0243  0.1558  0.0994
        96                0.0410  0.2026  0.1390
        168               0.0429  0.2071  0.1443
ES      24                0.0095  0.0976  0.0593
        96                0.0187  0.1369  0.0880
        168               0.0212  0.1456  0.0947
FR      24                0.0100  0.1001  0.0551
        96                0.0203  0.1425  0.0818
        168               0.0223  0.1494  0.0874
IT      24                0.0101  0.1007  0.0579
        96                0.0188  0.1373  0.0820
        168               0.0201  0.1418  0.0880